In [1]:
import pickle
import sys
import numpy as np
import pandas as pd
import artm
import seaborn as sns
import matplotlib.pyplot as plt
print artm.version()

from os import path, mkdir
from datetime import datetime
sys.path.insert(0, '..\\modules\\helpers')

import distances_helper as dh 
import create_model_helper as cmh

from plot_helper import PlotMaker
from config_helper import ConfigPaths


0.8.1


In [2]:
config = ConfigPaths('config.cfg')
plot_maker = PlotMaker()
print config.models_file_name

Q:\\topic_modeling\\csi_science_collections.git\experiments\UCI_filtered_ngramm_trimmed_without_names\np_28_02\models.txt


In [3]:
models_file = open(config.models_file_name, 'a')

In [258]:
def save_pickle_file(dists, filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'wb')
    pickle.dump(dists, pickle_file)
    pickle_file.close()
def load_pickle_file(filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'rb')
    p_file = pickle.load(pickle_file)
    pickle_file.close()
    return p_file
def save_model_pickle(_model_name, _model, _save=True):
    phi = _model.get_phi()
    phi = phi[(phi.T != 0).any()]
    theta = _model.get_theta()    
    saved_top_tokens = _model.score_tracker['top_tokens_score'].last_tokens
    if _save:
        save_pickle_file(phi, 'phi_{}.p'.format(_model_name))
        save_pickle_file(theta, 'theta_{}.p'.format(_model_name))
        save_pickle_file(saved_top_tokens, 'saved_top_tokens_{}.p'.format(_model_name))
    return phi, theta, saved_top_tokens
def load_model_pickle(_model_name, _distance_name):
    phi = load_pickle_file('phi_{}.p'.format(_model_name))
    theta = load_pickle_file('theta_{}.p'.format(_model_name))
    saved_top_tokens = load_pickle_file('saved_top_tokens_{}.p'.format(_model_name))
    distances = load_pickle_file('{}.p'.format(_distance_name))
    return phi, theta, saved_top_tokens, distances
# roman:  В каждой строчке Θ находим максимальное значение и в его столб-це остальных зануляем, а его делаем 1
def make_theta_for_single_decomposition(theta):
    theta_new = theta.copy()
    for row_idx, (row_name, row) in enumerate(theta_new.iterrows()):
        max_idx = row.idxmax()
        theta_new.iloc[:, max_idx] = 0
        theta_new.iloc[row_idx, max_idx] = 1
    return theta_new
# roman: В каждом столбце матрицы Φ зануляем n_topics − 1 самое маленькое значение
def make_phi_for_single_decomposition(phi, n_topics=None):
    if n_topics is None:
        n_topics = phi.shape[1]
    phi_new = phi.copy()
    for col_idx, (col_name, col) in enumerate(phi_new.iteritems()):
        th = col.sort_values()[n_topics] 
        col[col < th] = 0
        phi_new.iloc[:, col_idx] = phi_new.iloc[:, col_idx] / np.sum(phi_new.iloc[:, col_idx])
    return phi_new

# A = W x R; Lt={i:W_it != 0}, Rt={j: R_tj != 0}; 
# Разложение ! <-> не существует двух таких тем t1 и t2: Lt1 --- подмножество Lt2 или Rt1 --- подмножество Rt2
def check_theta_single_decomposition(theta, _debug_print=False):
    is_subset = lambda list_of_set, x: len([one_set for one_set in list_of_set if x.issubset(one_set) or one_set.issubset(x)]) != 0
    non_zero_indices = {}
    for row_idx, (_, row) in enumerate(theta.iterrows()):
        indices = set(np.where(row != 0)[0])
        if not is_subset(non_zero_indices.values(), indices):
            non_zero_indices[row_idx] = indices
        elif _debug_print:
            print 'Incorrect row index =', row_idx
    return len(non_zero_indices) == theta.shape[0]
def check_phi_single_decomposition(phi, _debug_print=False):
    is_subset = lambda list_of_set, x: len([one_set for one_set in list_of_set if x.issubset(one_set) or one_set.issubset(x)]) != 0
    non_zero_indices = {}
    for col_idx, (_, col) in enumerate(phi.iteritems()):
        indices = set(np.where(col != 0)[0])
        if not is_subset(non_zero_indices.values(), indices):
            non_zero_indices[col_idx] = indices
        elif _debug_print:
            print 'Incorrect column index =', col_idx
    return len(non_zero_indices) == phi.shape[1]

In [170]:
batch_vectorizer = artm.BatchVectorizer(data_path=config.output_batches_path,
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path=config.dictionary_path + '.dict')
n_iteration = 0

## Model 1

In [252]:
tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25, class_name='ngramm')
tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20, 
                              _model_name='model_100_1_iter_{}'.format(n_iteration))

name = model_100_1_iter_0, n_topics = 100, n_doc_passes = 5, seed_value = 100, n_iterations = 20, top_tokens_score = 15, topic_kernel_score = 0.25



In [253]:
phi, theta = tmp_model.get_phi(), tmp_model.get_theta()
phi_new = make_phi_for_single_decomposition(phi)
theta_new = make_theta_for_single_decomposition(theta)

print 'Check phi correctness = {}, theta correctness = {}' \
        .format(check_phi_single_decomposition(phi), check_theta_single_decomposition(theta))
print 'Check new_phi correctness = {}, new_theta correctness = {}' \
        .format(check_phi_single_decomposition(phi_new), check_theta_single_decomposition(theta_new))

Check phi correctness = True, theta correctness = False
Check new_phi correctness = True, new_theta correctness = True


## Model 7

In [254]:
tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25, class_name='ngramm')
tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', class_ids=['ngramm']))
tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['ngramm']))
tmp_model.regularizers['decorrelator_phi_regularizer'].tau = 10
tmp_model.regularizers['ss_theta_regularizer'].tau = -0.5
tmp_model.regularizers['ss_phi_regularizer'].tau = -0.5
tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20, 
                              _model_name='model_100_7_iter_{}'.format(n_iteration))

name = model_100_7_iter_0, n_topics = 100, n_doc_passes = 5, seed_value = 100, n_iterations = 20, top_tokens_score = 15, topic_kernel_score = 0.25
ss_theta_regularizer, tau = -0.5
decorrelator_phi_regularizer, tau = 10
ss_phi_regularizer, tau = -0.5



In [255]:
phi, theta = tmp_model.get_phi(), tmp_model.get_theta()
phi_new = make_phi_for_single_decomposition(phi)
theta_new = make_theta_for_single_decomposition(theta)

print 'Check phi correctness = {}, theta correctness = {}' \
        .format(check_phi_single_decomposition(phi), check_theta_single_decomposition(theta))
print 'Check new_phi correctness = {}, new_theta correctness = {}' \
        .format(check_phi_single_decomposition(phi_new), check_theta_single_decomposition(theta_new))

Check phi correctness = True, theta correctness = True
Check new_phi correctness = True, new_theta correctness = True


## Model 12

In [260]:
tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25, class_name='ngramm')
tmp_model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', class_ids=['ngramm']))
tmp_model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='ss_theta_regularizer'))
tmp_model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='ss_phi_regularizer', class_ids=['ngramm']))
tmp_model.regularizers['decorrelator_phi_regularizer'].tau = 10
tmp_model.regularizers['ss_theta_regularizer'].tau = -0.5
tmp_model.regularizers['ss_phi_regularizer'].tau = -2
tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                              tmp_model, _n_iterations=20, 
                              _model_name='model_100_12_iter_{}'.format(n_iteration))

name = model_100_12_iter_0, n_topics = 100, n_doc_passes = 5, seed_value = 100, n_iterations = 20, top_tokens_score = 15, topic_kernel_score = 0.25
ss_theta_regularizer, tau = -0.5
decorrelator_phi_regularizer, tau = 10
ss_phi_regularizer, tau = -2



In [261]:
phi, theta = tmp_model.get_phi(), tmp_model.get_theta()
phi_new = make_phi_for_single_decomposition(phi)
theta_new = make_theta_for_single_decomposition(theta)

print 'Check phi correctness = {}, theta correctness = {}' \
        .format(check_phi_single_decomposition(phi), check_theta_single_decomposition(theta))
print 'Check new_phi correctness = {}, new_theta correctness = {}' \
        .format(check_phi_single_decomposition(phi_new), check_theta_single_decomposition(theta_new))

Check phi correctness = True, theta correctness = True
Check new_phi correctness = True, new_theta correctness = True


Выбираем 3 модели: без регуляриз,  7 (норм, но у ядра размер 90) и 12 (норм и ядро размера 24). 

In [24]:
models_file.close()